In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [7]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = F.log_softmax(model(data), dim=1)
        loss = F.nll_loss(output, target) # nn.CrossEntropyLoss applies internally F.log_softmax and nn.NLLLoss
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = F.log_softmax(model(data), dim=1)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
torch.manual_seed(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../../../data', train=True, download=True,
                   transform=transform)
dataset2 = datasets.MNIST('../../../data', train=False,
                   transform=transform)

train_loader = torch.utils.data.DataLoader(dataset1, batch_size=64, num_workers=1, pin_memory=True, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=64, num_workers=1, pin_memory=True, shuffle=True)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, 10 + 1):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)
    scheduler.step()

# if args.save_model:
#     torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.317383
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.168252
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.279222
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.127341
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.135549
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.137428
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.083047
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.142610
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.033777
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.065821

Test set: Average loss: 0.0479, Accuracy: 9850/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.038757
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.016661
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.068970
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.144456
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.070102
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.021770
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.060962
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.088928
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.019470
T